In [3]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.parser import parse

In [266]:
#Read in the three CSV files

sp = "Data/S&P 500.csv"
mv = "Data/M2V (1).csv"
debt = "Data/US Debt.csv"

In [267]:
#Extract CSVs into dataframe
sp_df = pd.read_csv(sp)
mv_df = pd.read_csv(mv)
debt_df=pd.read_csv(debt)

In [268]:
debt_df

,DATE,GFDEBTN_20181129,GFDEBTN_20190410
0,1/1/66,320999,320999
1,4/1/66,316097,316097
2,7/1/66,324748,324748
3,10/1/66,329319,329319
4,1/1/67,330947,330947
5,4/1/67,322893,322893
6,7/1/67,335896,335896
7,10/1/67,344663,344663
8,1/1/68,349473,349473
9,4/1/68,345369,345369


In [269]:
#Transform the S&P data
sp_cols = ["Date","Adj Close", "Volume"]
sp_transformed = sp_df[sp_cols].copy()
sp_transformed = sp_transformed.rename(columns={"Date":"date",
                                               "Adj Close":"price",
                                               "Volume":"volume"})
sp_transformed.head()


,date,price,volume
0,1950-01-01,17.049999,42570000
1,1950-02-01,17.219999,33430000
2,1950-03-01,17.290001,40410000
3,1950-04-01,18.070000,48250000
4,1950-05-01,18.780001,45080000


In [270]:
mv_df.head()

,tt,M2V
0,1959-1-1,1.773
1,1959-4-1,1.789
2,1959-7-1,1.773
3,1959-10-1,1.779
4,1960-1-1,1.817


In [271]:
mv_transformed=mv_df.rename(columns={"tt":"date_unclean",
                                    "M2V":"velocity"})
mv_transformed.head()

,date_unclean,velocity
0,1959-1-1,1.773
1,1959-4-1,1.789
2,1959-7-1,1.773
3,1959-10-1,1.779
4,1960-1-1,1.817


In [272]:
mv_transformed["date"]=pd.to_datetime(mv_transformed["date_unclean"], format='%Y-%m-%d')
mv_final = mv_transformed[["date","velocity"]]
mv_final.head()

,date,velocity
0,1959-01-01,1.773
1,1959-04-01,1.789
2,1959-07-01,1.773
3,1959-10-01,1.779
4,1960-01-01,1.817


In [273]:
debt_df.head()

,DATE,GFDEBTN_20181129,GFDEBTN_20190410
0,1/1/66,320999,320999
1,4/1/66,316097,316097
2,7/1/66,324748,324748
3,10/1/66,329319,329319
4,1/1/67,330947,330947


In [274]:
debt_cols=["DATE", "GFDEBTN_20181129"]
debt_df2 = debt_df.iloc[16:]
debt_transformed=debt_df2[debt_cols].copy()
debt_transformed=debt_transformed.rename(columns={"DATE":"date_unclean",
                                                "GFDEBTN_20181129":"debt"})
debt_transformed.head()


,date_unclean,debt
16,1/1/70,372007
17,4/1/70,370094
18,7/1/70,378678
19,10/1/70,389158
20,1/1/71,391668


In [275]:
debt_transformed["date"]=pd.to_datetime(debt_transformed["date_unclean"], format = '%d/%m/%y')
#debt_transformed["date"] = debt_transformed["date1"] - pd.DateOffset(years=100)
#debt_final = debt_transformed[["date","debt"]]
#debt_final
debt_transformed.head()

,date_unclean,debt,date
16,1/1/70,372007,1970-01-01
17,4/1/70,370094,1970-01-04
18,7/1/70,378678,1970-01-07
19,10/1/70,389158,1970-01-10
20,1/1/71,391668,1971-01-01


In [276]:
debt_final=debt_transformed[["date","debt"]]
debt_final.head()

,date,debt
16,1970-01-01,372007
17,1970-01-04,370094
18,1970-01-07,378678
19,1970-01-10,389158
20,1971-01-01,391668


In [4]:
#Create connection
rds_connection_string = "root:password123@127.0.0.1/econ_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [5]:
engine.table_names()

['debt', 'sp', 'velocity']

In [279]:
mv_final.head()

,date,velocity
0,1959-01-01,1.773
1,1959-04-01,1.789
2,1959-07-01,1.773
3,1959-10-01,1.779
4,1960-01-01,1.817


In [280]:
#new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)
sp_transformed.to_sql(name="sp",con=engine, if_exists='append',index=False)


OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [281]:
mv_final.to_sql(name="velocity",con=engine, if_exists='append',index=False)

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [242]:
debt_final.to_sql(name="debt",con=engine, if_exists='append',index=False)


IntegrityError: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry '1970-01-01' for key 'PRIMARY'") [SQL: 'INSERT INTO debt (date, debt) VALUES (%s, %s)'] [parameters: ((datetime.datetime(1970, 1, 1, 0, 0), 372007), (datetime.datetime(1970, 1, 4, 0, 0), 370094), (datetime.datetime(1970, 1, 7, 0, 0), 378678), (datetime.datetime(1970, 1, 10, 0, 0), 389158), (datetime.datetime(1971, 1, 1, 0, 0), 391668), (datetime.datetime(1971, 1, 4, 0, 0), 397305), (datetime.datetime(1971, 1, 7, 0, 0), 412268), (datetime.datetime(1971, 1, 10, 0, 0), 424131)  ... displaying 10 of 196 total bound parameter sets ...  (datetime.datetime(2018, 1, 7, 0, 0), 21516058), (datetime.datetime(2018, 1, 10, 0, 0), 21974096))] (Background on this error at: http://sqlalche.me/e/gkpj)

In [ ]:
debt_final